In [2]:

#importing packages
import os
import cv2
import numpy as np
import pandas as pd

from tensorflow import keras
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.models import Sequential
from matplotlib import pyplot
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, precision_score
import seaborn as sns
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.models import Model
from keras.models import model_from_json
from keras.preprocessing import image
from tensorflow.keras import optimizers
from tensorflow.keras.applications import Xception
from keras import applications


ImportError: Traceback (most recent call last):
  File "C:\Users\MSI PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [ ]:
def load_subdata(data,SIZE):
    X1 = []
    y1 = []
    size = SIZE
    lbl = 0
    dic = {}
    for folder_name in os.listdir(data):
        
        Label = lbl
        dic[folder_name] = Label  
        count_img = 0;

        for filename in os.listdir(data + '/' + folder_name):
            image = tf.keras.preprocessing.image.load_img(data +'/'+ folder_name + '/' + filename,color_mode='rgb', target_size=(size,size))
            arr = keras.preprocessing.image.img_to_array(image)
            X1.append(np.array([arr]))
            y1.append(Label) 
        lbl +=1   
         
    X1 = np.asarray(X1).reshape(len(X1),SIZE,SIZE,3).astype(np.int16)
    y1 = np.asarray(y1).astype(np.uint8)
    return X1,y1, dic

In [ ]:
X, y, label_dic = load_subdata(r'../input/sign-language-dataset/Sign Language Dataset',224)

In [ ]:
from sklearn.utils import shuffle
X_new , y_new = shuffle(X, y, random_state=32)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_Test, y_train, y_Test = train_test_split(X_new , y_new, test_size=0.15, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_Test , y_Test, test_size=0.5, random_state=42)

In [ ]:
print("Training Images Shape (x train shape) :", X_train.shape)
print("Label of training images (y train shape) :",y_train.shape)

print("Validation Images Shape (x val shape) :",X_val.shape)
print("Label of Validation images (y val shape) :",y_val.shape)

print("Test Images Shape (x val shape) :",X_test.shape)
print("Label of Test images (y val shape) :",y_test.shape)

In [ ]:
y_trainHot = np.uint8(to_categorical(y_train, num_classes = 10))
y_testHot = np.uint8(to_categorical(y_test, num_classes = 10))
y_valHot = np.uint8(to_categorical(y_val, num_classes = 10))

In [ ]:
print("One hot encoded labels")
print(y_trainHot.shape)
print(y_testHot.shape)
print(y_valHot.shape)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i])
    
plt.show()

In [ ]:
def model_Evaluate(model):
    
    # Predict values for Test dataset
    y_pred = model.predict(X_test)
    y_test = y_testHot
    Y_pred = np.argmax(y_pred, axis=1)
    Y_Test = np.argmax(y_test, axis=1)
    
    class_names = list(label_dic.keys())
    print("Classification Report:")
    
    # Print the evaluation metrics for the dataset.
    print(classification_report(Y_Test, Y_pred, target_names=class_names))
    
    # Compute and plot the Confusion matrix
    cf_matrix = confusion_matrix(Y_Test, Y_pred)
    
    accuracy = recall_score(Y_Test, Y_pred, average=None)
    precision=precision_score(Y_Test,Y_pred,average=None)

    #class_names = class_names
    class_names = list(label_dic.keys())

    print("Confusion Matrix:")

    print(cf_matrix)
    ## Display the visualization of the Confusion Matrix.
    plt.show()
    
    print("Accuracy for each class: ")
    
    for i in range(len(accuracy)):
        print(f"{class_names[i]}:  {format(accuracy[i]*100, '.2f')}%")

In [ ]:
def plot_model(history):
    fig,(ax1,ax2)=plt.subplots(1,2,figsize=(12,4))
    fig.suptitle('Model Accuracy and Loss')

    ax1.plot(hist.history['accuracy'])
    ax1.plot(hist.history['val_accuracy'])
    ax1.title.set_text('Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train','Valid'],loc=4)

    ax2.plot(hist.history['loss'])
    ax2.plot(hist.history['val_loss'])
    ax2.title.set_text('Loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train','Valid'],loc=1)

    fig.show()
    

# Alexnet